In [ ]:
import os
import sys
from dotenv import load_dotenv

import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.auto import tqdm

load_dotenv()

# Set the download path for demonstration and ensure the folder exists.
download_path = os.getenv("ASA_DOWNLOAD_PATH")
os.makedirs(download_path, exist_ok=True)

git_path = os.getenv("GIT_FOLDER")
cv3_path = os.getenv("CV3_FOLDER")
sys.path.append(git_path)
sys.path.append(cv3_path)

In [73]:
from cerulean_cloud.cloud_function_asa.utils.analyzer import DarkAnalyzer

# Import functions from your script.
from asa_analysis.evaluation.source_analyzer_evaluation import (
    label_dark_vessel_results_with_distance,
    apply_labeling,
    process_groundtruth_on_analyzer,
)

In [ ]:
# Extracting values over iterations
import matplotlib.pyplot as plt


def plot_results_coin_search(example_search, search_values, decay_param="Radial Decay"):
    """
    Plot average coincidence scores for each dataset at each decay param
    """
    dark_truth = [entry["results_dark_truth"] for entry in example_search]
    dark_false = [entry["results_dark_false"] for entry in example_search]
    infra = [entry["results_infra"] for entry in example_search]
    fp = [entry["results_fp"] for entry in example_search]

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(search_values, dark_truth, label="Dark Truth", marker="^", color="green")
    plt.plot(search_values, dark_false, label="Dark False", marker="d", color="red")
    plt.plot(search_values, infra, label="Infra", marker="o", color="yellow")
    plt.plot(search_values, fp, label="FP", marker="s", color="orange")

    # Labels and title
    plt.xlabel(f"{decay_param} Values")
    plt.ylabel("Average Coincidence Scores")
    plt.xscale("log")  # Use log scale for better visibility if values are widely spaced
    plt.title(f"Coincidence scores vs. {decay_param} Values")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Show plot
    plt.show()


# Extracting values to plot
def plot_results_search_ratio(
    example_search,
    search_values,
    plot_just_sum=False,
    plot_just_product=False,
    diff_type="difference",
    decay_param="Radial Decay",
):
    """
    Plot ratios or differences of true and false coincidence scores
    """
    dark_truth = [entry["results_dark_truth"] for entry in example_search]
    dark_false = [entry["results_dark_false"] for entry in example_search]
    infra = [entry["results_infra"] for entry in example_search]
    fp = [entry["results_fp"] for entry in example_search]

    # Compute ratios (avoid division by zero)
    if diff_type == "ratio":
        diff_dark_truth_false = [
            dt / df if df != 0 else float("inf")
            for dt, df in zip(dark_truth, dark_false)
        ]
        diff_dark_truth_infra = [
            dt / i if i != 0 else float("inf") for dt, i in zip(dark_truth, infra)
        ]
        diff_dark_truth_fp = [
            dt / f if f != 0 else float("inf") for dt, f in zip(dark_truth, fp)
        ]

    if diff_type == "difference":
        diff_dark_truth_false = [dt - df for dt, df in zip(dark_truth, dark_false)]
        diff_dark_truth_infra = [dt - i for dt, i in zip(dark_truth, infra)]
        diff_dark_truth_fp = [dt - f for dt, f in zip(dark_truth, fp)]

    diff_sum = []
    for i in range(len(search_values)):
        diff_sum.append(diff_dark_truth_infra[i] + diff_dark_truth_fp[i])

    diff_product = []
    for i in range(len(search_values)):
        diff_product.append(diff_dark_truth_infra[i] * diff_dark_truth_fp[i])

    # Plotting
    plt.figure(figsize=(10, 6))
    if plot_just_product:
        plt.plot(
            search_values,
            diff_product,
            label="Product of infra and fp diffs",
            marker="o",
        )
    if plot_just_sum:
        plt.plot(search_values, diff_sum, label="Sum of infra and fp diffs", marker="o")
    if not plot_just_sum and not plot_just_product:
        plt.plot(
            search_values,
            diff_dark_truth_false,
            label="Dark Truth / Dark False",
            marker="o",
        )
        plt.plot(
            search_values, diff_dark_truth_infra, label="Dark Truth / Infra", marker="s"
        )
        plt.plot(search_values, diff_dark_truth_fp, label="Dark Truth / FP", marker="^")

    # Labels and title
    plt.xlabel(f"{decay_param} Values")
    plt.ylabel(f"Average {diff_type}")
    plt.xscale("log")  # Use log scale for better spacing
    plt.title(f"Dark Truth {diff_type} to Other Metrics")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Show plot
    plt.show()

In [ ]:
class DarkFinetuner:
    """
    A class to fine-tune detection parameters for dark vessel detection.

    Attributes:
        dark_groundtruth (GeoDataFrame): Ground truth data for dark vessels.
        infra_groundtruth (GeoDataFrame): Ground truth data for infrastructure.
        false_positives (GeoDataFrame): Data representing false positive detections.
        sar_detections (GeoDataFrame): SAR-based detections.
        analyzer_params (dict): Parameters for the analyzer.
        param_search (list): Stores search results for optimal parameters.
    """

    def __init__(
        self,
        dark_groundtruth,
        infra_groundtruth,
        false_positives,
        sar_detections,
        starting_params,
    ):
        self.dark_groundtruth = dark_groundtruth
        self.infra_groundtruth = infra_groundtruth
        self.false_positives = false_positives
        self.sar_detections = sar_detections
        self.analyzer_params = starting_params

        self.results_dark_truth = None
        self.results_dark_false = None
        self.results_infra = None
        self.results_fp = None

        self.coincidence_means = {
            "results_dark_truth": 0,
            "results_dark_false": 0,
            "results_infra": 0,
            "results_fp": 0,
        }

        self.param_search = []

    def highest_coincidence_mean(self, results_gdf):
        """
        Computes the highest coincidence mean by selecting the maximum coincidence_score per slick_id.

        Args:
            results_gdf (GeoDataFrame): Results data containing coincidence scores.

        Returns:
            float: The mean of the highest coincidence scores per slick_id.
        """
        scores = [
            g.sort_values(by="coincidence_score", ascending=False).iloc[0][
                "coincidence_score"
            ]
            for _, g in results_gdf.groupby(by="slick_id")
            if len(g) > 0
        ]
        return sum(scores) / len(scores) if scores else 0

    def generate_results(self):
        """
        Processes ground truth datasets through the DarkAnalyzer and computes coincidence means.
        """
        results_dark = process_groundtruth_on_analyzer(
            DarkAnalyzer,
            self.dark_groundtruth,
            points_gdf=self.sar_detections,
            analyzer_params=self.analyzer_params,
        )
        results_dark = apply_labeling(
            results_dark, self.dark_groundtruth, label_dark_vessel_results_with_distance
        )
        self.results_dark_truth = results_dark[results_dark["truth"]]
        self.results_dark_false = results_dark[~results_dark["truth"]]

        self.results_infra = process_groundtruth_on_analyzer(
            DarkAnalyzer,
            self.infra_groundtruth,
            points_gdf=self.sar_detections,
            analyzer_params=self.analyzer_params,
            post_filter_dark=True,
        )
        self.results_fp = process_groundtruth_on_analyzer(
            DarkAnalyzer,
            self.false_positives,
            points_gdf=self.sar_detections,
            analyzer_params=self.analyzer_params,
            post_filter_dark=True,
        )

        self.coincidence_means = {
            "results_dark_truth": self.highest_coincidence_mean(
                self.results_dark_truth
            ),
            "results_dark_false": self.highest_coincidence_mean(
                self.results_dark_false
            ),
            "results_infra": self.highest_coincidence_mean(self.results_infra),
            "results_fp": self.highest_coincidence_mean(self.results_fp),
        }

    def find_optimal_search_value(
        self,
        param_search,
        search_values,
        do_product=False,
        do_sum=True,
        diff_type="ratio",
    ):
        """
        Finds the optimal parameter value by comparing dark vessel detection performance.

        Args:
            param_search (list): List of dictionaries containing coincidence means.
            search_values (list): List of parameter values.
            do_product (bool, optional): Whether to use the product of differences. Defaults to False.
            do_sum (bool, optional): Whether to use the sum of differences. Defaults to True.
            diff_type (str, optional): Method to compute differences ('ratio' or 'difference'). Defaults to 'ratio'.

        Returns:
            tuple: Index and value of the best parameter.
        """
        dark_truth = [entry["results_dark_truth"] for entry in param_search]
        infra = [entry["results_infra"] for entry in param_search]
        fp = [entry["results_fp"] for entry in param_search]

        if diff_type == "ratio":
            diff_dark_truth_infra = [
                dt / i if i != 0 else float("inf") for dt, i in zip(dark_truth, infra)
            ]
            diff_dark_truth_fp = [
                dt / f if f != 0 else float("inf") for dt, f in zip(dark_truth, fp)
            ]
        else:
            diff_dark_truth_infra = [dt - i for dt, i in zip(dark_truth, infra)]
            diff_dark_truth_fp = [dt - f for dt, f in zip(dark_truth, fp)]

        if do_sum:
            diff_sums = [
                diff_dark_truth_infra[i] + diff_dark_truth_fp[i]
                for i in range(len(search_values))
            ]
            return np.argmax(diff_sums), search_values[np.argmax(diff_sums)]
        if do_product:
            diff_products = [
                diff_dark_truth_infra[i] * diff_dark_truth_fp[i]
                for i in range(len(search_values))
            ]
            return np.argmax(diff_products), search_values[np.argmax(diff_products)]

    def perform_param_search(self, parameter, search_values):
        """
        Iterates over a set of search values, evaluating detection performance for each parameter.

        Args:
            parameter (str): The parameter to be optimized.
            search_values (list): A list of values to test.
        """
        params_search = []
        for value in tqdm(search_values, desc=f"Params searching {parameter}..."):
            self.analyzer_params[parameter] = value
            self.generate_results()
            params_search.append(self.coincidence_means.copy())
        self.param_search = params_search

In [ ]:
collation_datasets_folder = cv3_path + "/asa_analysis/evaluation/"
dark_vess_df = pd.read_csv(
    f"{collation_datasets_folder}/refined_dark_vessel_dataset.csv"
)
sar_detections = pd.read_csv(
    f"{collation_datasets_folder}/sar_detections_hitl_dark_ds.csv"
)
sar_detections_infra_vess = pd.read_csv(
    f"{collation_datasets_folder}/gfw_sar_detections_for_hitl.csv"
).drop(columns="Unnamed: 0")

dark_vessel_groundtruth = gpd.GeoDataFrame(
    dark_vess_df,
    geometry=gpd.points_from_xy(dark_vess_df["lon"], dark_vess_df["lat"]),
    crs="EPSG:4326",
)
sar_detections_gdf = gpd.GeoDataFrame(
    sar_detections,
    geometry=gpd.points_from_xy(
        sar_detections["detect_lon"], sar_detections["detect_lat"]
    ),
    crs="EPSG:4326",
)
sar_detections_gdf = sar_detections_gdf[sar_detections_gdf["structure_id"].isna()]
sar_detections_gdf = sar_detections_gdf.reset_index()

sar_detections_infra_vess_gdf = gpd.GeoDataFrame(
    sar_detections_infra_vess,
    geometry=gpd.points_from_xy(
        sar_detections_infra_vess["detect_lon"], sar_detections_infra_vess["detect_lat"]
    ),
    crs="EPSG:4326",
)
sar_detections_infra_vess_gdf = sar_detections_infra_vess_gdf[
    sar_detections_infra_vess_gdf["structure_id"].isna()
]
sar_detections_infra_vess_gdf = sar_detections_infra_vess_gdf.reset_index()
# Load hitl CSV for vessel and infrastructure groundtruth.
hitl_df = pd.read_csv(f"{collation_datasets_folder}/slick_to_source_2025-3-20.csv")
infrastructure_groundtruth = hitl_df[
    (hitl_df["type"] == 2) & (hitl_df["hitl_verification"])
]
if "slick" in infrastructure_groundtruth.columns:
    infrastructure_groundtruth = infrastructure_groundtruth.rename(
        columns={"slick": "slick_id"}
    )

reviewed_fp = pd.read_csv(
    cv3_path
    + r"\asa_analysis\false positive\HITL Source Dataset ASA - False Positives.csv"
)
reviewed_fp = reviewed_fp[reviewed_fp["Marked As False"] == "y"]
reviewed_fp = reviewed_fp.rename(
    columns={"Scene ID": "scene_id", "Slick ID": "slick_id"}
)


fp_sar = pd.read_csv(
    cv3_path + r"\asa_analysis\dark vessels\false_positives_sar_detections.csv"
)
fp_sar_gdf = gpd.GeoDataFrame(
    fp_sar,
    geometry=gpd.points_from_xy(fp_sar["detect_lon"], fp_sar["detect_lat"]),
    crs="EPSG:4326",
)
fp_sar_gdf = fp_sar_gdf[fp_sar_gdf["structure_id"].isna()]
fp_sar_gdf = fp_sar_gdf.reset_index()

sar_detections = pd.concat(
    [fp_sar_gdf, sar_detections_infra_vess_gdf, sar_detections_gdf]
)

In [ ]:
starting_params = {
    "decay_theta": 10.0,
    "decay_radius": 15000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_radius"
search_values = [100, 1000, 2000, 5000, 10000, 20000, 50000]
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
example_search = [
    {
        "results_dark_truth": 2.5865327964462703e-05,
        "results_dark_false": 4.23078638783195e-06,
        "results_infra": 0.0007623858410658746,
        "results_fp": 3.275326148898183e-05,
    },
    {
        "results_dark_truth": 0.05807653863532608,
        "results_dark_false": 0.010543061136581345,
        "results_infra": 0.035429948030474494,
        "results_fp": 0.00837103008916577,
    },
    {
        "results_dark_truth": 0.16176527658162365,
        "results_dark_false": 0.026169144750124246,
        "results_infra": 0.060965680470452775,
        "results_fp": 0.017351994090412874,
    },
    {
        "results_dark_truth": 0.3533692150067619,
        "results_dark_false": 0.07061814011574402,
        "results_infra": 0.11005378806559986,
        "results_fp": 0.04649152521828429,
    },
    {
        "results_dark_truth": 0.49183275255878345,
        "results_dark_false": 0.13810361989809267,
        "results_infra": 0.16718821590526914,
        "results_fp": 0.09393747081233408,
    },
    {
        "results_dark_truth": 0.6022057236385109,
        "results_dark_false": 0.23486556136929354,
        "results_infra": 0.24589631139307622,
        "results_fp": 0.1708196276695335,
    },
    {
        "results_dark_truth": 0.6960849796810236,
        "results_dark_false": 0.38278706495836545,
        "results_infra": 0.37524835663649775,
        "results_fp": 0.2963864326508636,
    },
]

In [ ]:
plot_results_coin_search(example_search, search_values)

In [ ]:
plot_results_search_ratio(example_search, search_values, decay_param="Radial Decay")

In [ ]:
plot_results_search_ratio(
    example_search, search_values, plot_just_sum=True, diff_type="difference"
)

In [ ]:
dark_finetuner.find_optimal_search_value(
    example_search, search_values, diff_type="difference"
)

In [ ]:
starting_params = {
    "decay_theta": 10.0,
    "decay_radius": 20000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_theta"
search_values = [0.0, 0.5, 1.0, 10.0, 20.0, 40.0, 80.0]
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.7939771346342847,
        "results_dark_false": 0.6045750363070851,
        "results_infra": 0.7095622014157633,
        "results_fp": 0.4539167077779148,
    },
    {
        "results_dark_truth": 0.7818155270917301,
        "results_dark_false": 0.5101618586860095,
        "results_infra": 0.6170405628873684,
        "results_fp": 0.39782169138433554,
    },
    {
        "results_dark_truth": 0.7699845452422146,
        "results_dark_false": 0.45202390097785256,
        "results_infra": 0.555497219807682,
        "results_fp": 0.3602810232896382,
    },
    {
        "results_dark_truth": 0.6022057236385109,
        "results_dark_false": 0.23486556136929354,
        "results_infra": 0.24589631139307622,
        "results_fp": 0.1708196276695335,
    },
    {
        "results_dark_truth": 0.4807650816080745,
        "results_dark_false": 0.17827240772196212,
        "results_infra": 0.1690872524090447,
        "results_fp": 0.12830803752189568,
    },
    {
        "results_dark_truth": 0.3335322713669502,
        "results_dark_false": 0.12915426903066443,
        "results_infra": 0.11222953793210899,
        "results_fp": 0.09404573602286316,
    },
    {
        "results_dark_truth": 0.19857592898747103,
        "results_dark_false": 0.09390504601189258,
        "results_infra": 0.07179463440769009,
        "results_fp": 0.06446787505507375,
    },
]

In [ ]:
plot_results_coin_search(example_search, search_values)

In [ ]:
plot_results_search_ratio(example_search, search_values, decay_param="Theta Decay")

In [ ]:
plot_results_search_ratio(
    example_search, search_values, decay_param="Theta Decay", plot_just_sum=True
)

In [ ]:
starting_params = {
    "decay_theta": 10.0,
    "decay_radius": 15000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_radius"
search_values = np.linspace(10000, 50000, 13)
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
# dark_finetuner.param_search

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.49183275255878345,
        "results_dark_false": 0.13810361989809267,
        "results_infra": 0.16718821590526914,
        "results_fp": 0.09393747081233408,
    },
    {
        "results_dark_truth": 0.5416047757574964,
        "results_dark_false": 0.17411718497086714,
        "results_infra": 0.19746333304338834,
        "results_fp": 0.12270102105245516,
    },
    {
        "results_dark_truth": 0.5760677606344763,
        "results_dark_false": 0.206230329196251,
        "results_infra": 0.2232444754144052,
        "results_fp": 0.1483386605662648,
    },
    {
        "results_dark_truth": 0.6022057236385109,
        "results_dark_false": 0.23486556136929354,
        "results_infra": 0.24589631139307622,
        "results_fp": 0.1708196276695335,
    },
    {
        "results_dark_truth": 0.6222863481732138,
        "results_dark_false": 0.25994100898840894,
        "results_infra": 0.2663716520891139,
        "results_fp": 0.19061959391820485,
    },
    {
        "results_dark_truth": 0.6382015210847589,
        "results_dark_false": 0.2818704066120561,
        "results_infra": 0.2850169911487418,
        "results_fp": 0.20850795390351917,
    },
    {
        "results_dark_truth": 0.6511273963392981,
        "results_dark_false": 0.3009758746411586,
        "results_infra": 0.30200577288651437,
        "results_fp": 0.22506619199561306,
    },
    {
        "results_dark_truth": 0.6618346287041108,
        "results_dark_false": 0.3178573836329363,
        "results_infra": 0.3173038372968445,
        "results_fp": 0.24005028524723526,
    },
    {
        "results_dark_truth": 0.6708496154056024,
        "results_dark_false": 0.33306024758604347,
        "results_infra": 0.3312223095420399,
        "results_fp": 0.2534562972928998,
    },
    {
        "results_dark_truth": 0.6785442989186558,
        "results_dark_false": 0.34712163314640615,
        "results_infra": 0.343753306855129,
        "results_fp": 0.26562171238523447,
    },
    {
        "results_dark_truth": 0.6851890017461857,
        "results_dark_false": 0.3601657419589784,
        "results_infra": 0.35522535345278433,
        "results_fp": 0.2767959040889918,
    },
    {
        "results_dark_truth": 0.6909849373115583,
        "results_dark_false": 0.37199055551359883,
        "results_infra": 0.3656540091056572,
        "results_fp": 0.2870182975434919,
    },
    {
        "results_dark_truth": 0.6960849796810236,
        "results_dark_false": 0.38278706495836545,
        "results_infra": 0.37524835663649775,
        "results_fp": 0.2963864326508636,
    },
]

In [ ]:
plot_results_coin_search(example_search, search_values)

In [ ]:
plot_results_search_ratio(example_search, search_values, diff_type="difference")

In [ ]:
search_values

In [ ]:
starting_params = {
    "decay_theta": 10.0,
    "decay_radius": 23000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_theta"
search_values = np.linspace(1.0, 20.0, 11)
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
# dark_finetuner.param_search

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.7913640864065017,
        "results_dark_false": 0.481717967735901,
        "results_infra": 0.5795076121351173,
        "results_fp": 0.39311885061245505,
    },
    {
        "results_dark_truth": 0.7482693159543223,
        "results_dark_false": 0.38328571036704157,
        "results_infra": 0.44062519453085947,
        "results_fp": 0.3008100955892892,
    },
    {
        "results_dark_truth": 0.7091880420241673,
        "results_dark_false": 0.32933494087175136,
        "results_infra": 0.36813901675484106,
        "results_fp": 0.2532802921014842,
    },
    {
        "results_dark_truth": 0.6736043535212421,
        "results_dark_false": 0.2943743960706704,
        "results_infra": 0.31991002510563393,
        "results_fp": 0.22237363771528657,
    },
    {
        "results_dark_truth": 0.6421245558942146,
        "results_dark_false": 0.2711897369042783,
        "results_infra": 0.2846141986512288,
        "results_fp": 0.2005655392956427,
    },
    {
        "results_dark_truth": 0.6130597427845297,
        "results_dark_false": 0.253181867475079,
        "results_infra": 0.2581056620270646,
        "results_fp": 0.1851012978533988,
    },
    {
        "results_dark_truth": 0.5861440746528404,
        "results_dark_false": 0.23826202657436293,
        "results_infra": 0.23725616963616802,
        "results_fp": 0.17315698921216482,
    },
    {
        "results_dark_truth": 0.5611517192890445,
        "results_dark_false": 0.22536682316959558,
        "results_infra": 0.2206441574316884,
        "results_fp": 0.16358898624033968,
    },
    {
        "results_dark_truth": 0.5378892779202061,
        "results_dark_false": 0.21452336966692626,
        "results_infra": 0.20671276540402247,
        "results_fp": 0.15540044844568335,
    },
    {
        "results_dark_truth": 0.5161898519402516,
        "results_dark_false": 0.20561688188665994,
        "results_infra": 0.1947087762252549,
        "results_fp": 0.1483639079222767,
    },
    {
        "results_dark_truth": 0.4959083695835289,
        "results_dark_false": 0.19778000449952324,
        "results_infra": 0.18429747356921544,
        "results_fp": 0.14231401172139377,
    },
]

In [ ]:
plot_results_coin_search(example_search, search_values, decay_param="Theta Decay")

In [ ]:
plot_results_search_ratio(example_search, search_values, decay_param="Theta Decay")

In [ ]:
dark_finetuner.find_optimal_search_value(
    example_search, search_values, diff_type="difference"
)

In [ ]:
starting_params = {
    "decay_theta": 6.7,
    "decay_radius": 15000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_radius"
search_values = np.linspace(15000, 25000, 11)
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
# dark_finetuner.param_search

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.6108680359490063,
        "results_dark_false": 0.2248995608785922,
        "results_infra": 0.25982092926263695,
        "results_fp": 0.16020426228982998,
    },
    {
        "results_dark_truth": 0.6213285213012711,
        "results_dark_false": 0.2348832218838559,
        "results_infra": 0.2686385611588727,
        "results_fp": 0.16916158828893427,
    },
    {
        "results_dark_truth": 0.6308334218463962,
        "results_dark_false": 0.24438770431395654,
        "results_infra": 0.27701706212770494,
        "results_fp": 0.17775497859266784,
    },
    {
        "results_dark_truth": 0.6395089434319893,
        "results_dark_false": 0.2535344968588574,
        "results_infra": 0.2850622242951687,
        "results_fp": 0.18598122850276524,
    },
    {
        "results_dark_truth": 0.6474598076157365,
        "results_dark_false": 0.26232798959811027,
        "results_infra": 0.2926861625357562,
        "results_fp": 0.1938773539470957,
    },
    {
        "results_dark_truth": 0.6547736810545907,
        "results_dark_false": 0.27066289889963724,
        "results_infra": 0.2999790121938222,
        "results_fp": 0.20146069582132,
    },
    {
        "results_dark_truth": 0.6615245415223817,
        "results_dark_false": 0.2787098505537742,
        "results_infra": 0.30693673098771496,
        "results_fp": 0.2087133132979035,
    },
    {
        "results_dark_truth": 0.6677752698170953,
        "results_dark_false": 0.2867266387979754,
        "results_infra": 0.3135510390053842,
        "results_fp": 0.21565213916010215,
    },
    {
        "results_dark_truth": 0.6736043535212421,
        "results_dark_false": 0.2943743960706704,
        "results_infra": 0.31991002510563393,
        "results_fp": 0.22237363771528654,
    },
    {
        "results_dark_truth": 0.6792360892174635,
        "results_dark_false": 0.3018417038054928,
        "results_infra": 0.32603813576212787,
        "results_fp": 0.22880872463717636,
    },
    {
        "results_dark_truth": 0.6844973879689307,
        "results_dark_false": 0.30912533551454346,
        "results_infra": 0.3319550636851774,
        "results_fp": 0.2349737630051403,
    },
]

In [ ]:
plot_results_search_ratio(example_search, search_values, diff_type="difference")

In [ ]:
dark_finetuner.find_optimal_search_value(
    example_search, search_values, diff_type="difference"
)

In [ ]:
search_values

In [ ]:
starting_params = {
    "decay_theta": 10.0,
    "decay_radius": 19000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_theta"
search_values = np.linspace(4.0, 12, 11)
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
# dark_finetuner.param_search

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.6974767741333314,
        "results_dark_false": 0.3142290833326668,
        "results_infra": 0.36421840295895774,
        "results_fp": 0.23627431183456674,
    },
    {
        "results_dark_truth": 0.6819636812738608,
        "results_dark_false": 0.296070390374665,
        "results_infra": 0.3383378830788131,
        "results_fp": 0.22094261316989203,
    },
    {
        "results_dark_truth": 0.6670499725297115,
        "results_dark_false": 0.28020666884567397,
        "results_infra": 0.3171829698651755,
        "results_fp": 0.2083732443027372,
    },
    {
        "results_dark_truth": 0.6527013776064962,
        "results_dark_false": 0.26681305885485623,
        "results_infra": 0.29895752092147015,
        "results_fp": 0.19756824252889102,
    },
    {
        "results_dark_truth": 0.6388862339786714,
        "results_dark_false": 0.2552360866609884,
        "results_infra": 0.28289453465017467,
        "results_fp": 0.1881864068806723,
    },
    {
        "results_dark_truth": 0.6255752504237613,
        "results_dark_false": 0.2456033315681641,
        "results_infra": 0.26858937414200923,
        "results_fp": 0.1801592083929998,
    },
    {
        "results_dark_truth": 0.6129592574139748,
        "results_dark_false": 0.23737738694847393,
        "results_infra": 0.25588646201816334,
        "results_fp": 0.17317994960879676,
    },
    {
        "results_dark_truth": 0.6009637026593643,
        "results_dark_false": 0.23010868160328177,
        "results_infra": 0.24458509818544,
        "results_fp": 0.16713302239637318,
    },
    {
        "results_dark_truth": 0.5893591427909091,
        "results_dark_false": 0.22344027272839637,
        "results_infra": 0.2344418121940409,
        "results_fp": 0.16174370270178773,
    },
    {
        "results_dark_truth": 0.5781270152649428,
        "results_dark_false": 0.2173978483291584,
        "results_infra": 0.2252484693566563,
        "results_fp": 0.1568637223630215,
    },
    {
        "results_dark_truth": 0.5672499917529706,
        "results_dark_false": 0.21181509928743614,
        "results_infra": 0.2170110963437772,
        "results_fp": 0.15235066792388452,
    },
]

In [ ]:
plot_results_search_ratio(example_search, search_values, decay_param="Theta Decay")

In [ ]:
plot_results_search_ratio(
    example_search, search_values, plot_just_sum=True, decay_param="Theta Decay"
)

In [ ]:
plot_results_search_ratio(
    example_search, search_values, decay_param="Theta Decay", plot_just_product=True
)

In [ ]:
dark_finetuner.find_optimal_search_value(
    example_search, search_values, diff_type="difference"
)

In [ ]:
starting_params = {
    "decay_theta": 6.4,
    "decay_radius": 15000,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}

dark_finetuner = DarkFinetuner(
    dark_groundtruth=dark_vessel_groundtruth,
    infra_groundtruth=infrastructure_groundtruth,
    false_positives=reviewed_fp,
    sar_detections=sar_detections,
    starting_params=starting_params,
)

parameter = "decay_radius"
search_values = np.linspace(16000, 24000, 11)
# dark_finetuner.perform_param_search(parameter, search_values)
# clear_output()

In [ ]:
# dark_finetuner.param_search

In [ ]:
example_search = [
    {
        "results_dark_truth": 0.6263984343225979,
        "results_dark_false": 0.23916017350276642,
        "results_infra": 0.2745414249448279,
        "results_fp": 0.17234896575415656,
    },
    {
        "results_dark_truth": 0.6341230740526299,
        "results_dark_false": 0.2469071411935572,
        "results_infra": 0.28137732295556844,
        "results_fp": 0.17940620716856917,
    },
    {
        "results_dark_truth": 0.6412985945956257,
        "results_dark_false": 0.25435019564686323,
        "results_infra": 0.28795909404826525,
        "results_fp": 0.18621425876294995,
    },
    {
        "results_dark_truth": 0.6479819505622778,
        "results_dark_false": 0.2615130688988036,
        "results_infra": 0.29434318794135017,
        "results_fp": 0.1927811589204094,
    },
    {
        "results_dark_truth": 0.6542224161736243,
        "results_dark_false": 0.2685455687682306,
        "results_infra": 0.3004615645152379,
        "results_fp": 0.1991354194254741,
    },
    {
        "results_dark_truth": 0.6600628500805591,
        "results_dark_false": 0.27529345726399984,
        "results_infra": 0.30633454104614705,
        "results_fp": 0.205264967049702,
    },
    {
        "results_dark_truth": 0.6655407156767298,
        "results_dark_false": 0.28176231081721786,
        "results_infra": 0.3120139215798789,
        "results_fp": 0.21119022628358716,
    },
    {
        "results_dark_truth": 0.6706889108825708,
        "results_dark_false": 0.28799352631735536,
        "results_infra": 0.3174942702502546,
        "results_fp": 0.21693583015074164,
    },
    {
        "results_dark_truth": 0.6755364480467193,
        "results_dark_false": 0.2942831022661865,
        "results_infra": 0.3227492532180434,
        "results_fp": 0.22248221999741666,
    },
    {
        "results_dark_truth": 0.6801090149043367,
        "results_dark_false": 0.30040963324392284,
        "results_infra": 0.3278221685337474,
        "results_fp": 0.2278386359400121,
    },
    {
        "results_dark_truth": 0.6844824978471229,
        "results_dark_false": 0.30631207816249467,
        "results_infra": 0.33274009796345005,
        "results_fp": 0.2330396888246176,
    },
]

In [ ]:
plot_results_search_ratio(example_search, search_values, diff_type="difference")

In [ ]:
search_values

In [ ]:
plot_results_search_ratio(
    example_search, search_values, diff_type="difference", plot_just_sum=True
)

In [ ]:
dark_finetuner.find_optimal_search_value(
    example_search, search_values, diff_type="difference"
)

In [ ]:
# Final fine-tuned params
finetuned_params = {
    "decay_theta": 6.4,
    "decay_radius": 19200,
    "endpoints_offset": 0.05,
    "endpoints_gap": 0.05,
}